In [88]:
# Import all required libraries
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('data/X_train.txt', delimiter=None) 
Y = np.genfromtxt('data/Y_train.txt', delimiter=None) 
X,Y = ml.shuffleData(X,Y)
Xtr, Xva, Ytr, Yva = ml.splitData(X,Y,0.75)
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

In [50]:
a= [1,2,3]
b = np.ones(len(a))
print(b)
print(np.vstack((b-a, a)).T)


[ 1.  1.  1.]
[[ 0.  1.]
 [-1.  2.]
 [-2.  3.]]


In [83]:
boosts = []
n_boosts = 5000

Ytr_ = np.copy(Ytr)  # We're going to copy the data becuase each booster iteration we're going to mess with it.
#print(Ytr_.shape)
for i in range(n_boosts):
    tree = ml.dtree.treeRegress(Xtr, Ytr_, maxDepth=2)
    """you can substitute it with linear regression"""
    boosts.append(tree)
    # Now "learning" from out mistakes.
    Ytr_ -= tree.predict(Xtr)
    #print(Ytr_)

In [84]:
class Gradient_Boosted_Tree(ml.base.classifier):
    def __init__(self, boosts):
        """Constructs a Gradient_boosted_Tree class with a set of boosts. """
        self.boosts = boosts
    
    def predictSoft(self, X):
        """Predicts regression values using boosting. """
        n_boosts = len(self.boosts)
        preds = [self.boosts[i].predict(X) for i in range(n_boosts)]
        #print(preds)
        p1 = np.sum(preds, axis=0)
        p0 = np.ones(len(p1)) - p1
        pre = np.vstack((p0, p1)).T
        return pre


In [85]:
gbt = Gradient_Boosted_Tree(boosts)
gbt.classes = np.unique(Y)

probs = gbt.predictSoft(Xte)
print(probs)
print("{0:>15}: {1:.4f}".format('Train AUC', gbt.auc(Xtr, Ytr)))
print("{0:>15}: {1:.4f}".format('Validation AUC', gbt.auc(Xva, Yva)))

[[ 0.26497594  0.73502406]
 [ 0.41760941  0.58239059]
 [-0.00420907  1.00420907]
 ..., 
 [ 0.70273874  0.29726126]
 [ 1.02929083 -0.02929083]
 [ 0.8272334   0.1727666 ]]
      Train AUC: 0.8726
 Validation AUC: 0.7699


In [92]:
Y1test = gbt.predictSoft(Xte)[:,1]


TypeError: 'int' object is not iterable

In [93]:
for i in range(len(Y1test)):
    if Y1test[i] < 0:
        Y1test[i] = 0
    if Y1test[i] > 1:
        Y1test[i] = 1
Yte = np.vstack((np.arange(Xte.shape[0]), Y1test)).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit2.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')